In [1]:
import pprint
import time

import zmq

ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
pupil_remote = ctx.socket(zmq.REQ)

ip = 'localhost'  # If you talk to a different machine use its IP.
port = 50020  # The port defaults to 50020. Set in Pupil Capture GUI.

pupil_remote.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
pupil_remote.send_string('SUB_PORT')
sub_port = pupil_remote.recv_string()

# Request 'PUB_PORT' for writing data
pupil_remote.send_string('PUB_PORT')
pub_port = pupil_remote.recv_string()

In [2]:
import requests

def get_object_in_gaze(gaze, r):
    for result in r:
        boxes = result.boxes
        classes = boxes.cls
        for i, box in enumerate(boxes.xyxyn):
            # Check if the gaze point lies within the bounding box of detected objects
            if box[0] <= gaze[0] <= box[2] and box[1] <= (1 - gaze[1]) <= box[3]:
                return result.names[int(classes[i])]
    return None

def switch_lamp_on():
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    
    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)
    
        # Check if the request was successful
        if response.status_code == 200:
            # Print the Thing Description in JSON format
            td = response.json()
            print(td["on"])

            # Define the payload to turn the lamp off
            payload = {
                "on": False if td["on"] else True
            }
            
            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print("The lamp has been switches.")
                else:
                    print(f"Failed to turn off the lamp. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        
def set_lamp_brightness(lamp_brighness=100):
    if lamp_brighness < 0 or lamp_brighness > 100:
        print("Brightness value must be between 0 and 100.")
        return
    
    # URL for the Thing Description (base URL provided in the TD itself)
    td_url = "http://10.2.2.33:1880/r402/theglobe"
    
    try:
        # Send a GET request to retrieve the Thing Description
        response = requests.get(td_url)
    
        # Check if the request was successful
        if response.status_code == 200:
            # Define the payload to turn the lamp off
            payload = {
                "brightness": lamp_brighness
            }
            
            # Send the PUT request to change the state of the lamp
            try:
                response = requests.put(td_url, json=payload)
                if response.status_code == 200:
                    print(f"Lamp brightness has been set to {lamp_brighness}.")
                else:
                    print(f"Failed to turn off the lamp. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
        else:
            print(f"Failed to retrieve Thing Description. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

def detect_camera_movement(prev_frame, curr_frame, tr):
    # Convert frames to grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Parameters for optical flow (Lucas-Kanade method)
    lk_params = dict(winSize=(21, 21), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

    # Detect good features to track in the first frame
    prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7))

    # Calculate optical flow (track points from prev_frame to curr_frame)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_points, None, **lk_params)

    # Select good points (where the flow has been successfully found)
    good_prev_points = prev_points[status == 1]
    good_next_points = next_points[status == 1]

    # Calculate the displacement of points
    displacements = good_next_points - good_prev_points
    avg_displacement = displacements.mean(axis=0)

    # Direction based on average displacement
    dx, dy = avg_displacement
    
    # Return True if the movement is greater than the threshold
    if abs(dx) > tr or abs(dy) > tr:
        return True
    
    return False
    
def unpack_frame_img(d):
    fr = d[2]
    fr = cv2.imdecode(np.frombuffer(fr, dtype=np.uint8), cv2.IMREAD_COLOR)
    # Convert the image to RGB
    fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(fr)
    

In [3]:
from ultralytics import YOLO

# Setup ZeroMQ context and subscriber socket
ctx = zmq.Context()

def create_socket(ctx_c, ip_c, topics):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{sub_port}')
    for topic in topics:
        sub.subscribe(topic)
    return sub

def create_socket_topic(ctx_c, ip_c):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{sub_port}')
    sub.subscribe("frame.world")
    return sub

def create_socket_gaze(ctx_c, ip_c):
    sub = ctx_c.socket(zmq.SUB)
    sub.connect(f'tcp://{ip_c}:{sub_port}')
    sub.subscribe("gaze")
    return sub

# Load a model
model = YOLO("../.local/models/object_detection/yolov11n_trained.pt")  # pretrained YOLOv8n model

In [ ]:
import msgpack
import time
import numpy as np
import cv2
from PIL import Image

sub_frame = create_socket(ctx, ip, ["frame.world"])
last_frame = sub_frame.recv_multipart()
results = model(unpack_frame_img(last_frame))
sub_frame.close()

sub_fixations = create_socket(ctx, ip, ["fixations"])

while True:
    time.sleep(0.5)
    sub_frame = create_socket(ctx, ip, ["frame.world"])
    frame = sub_frame.recv_multipart()
    sub_frame.close()
    
    im1 = cv2.imdecode(np.frombuffer(last_frame[2], dtype=np.uint8), cv2.IMREAD_COLOR)
    im2 = cv2.imdecode(np.frombuffer(frame[2], dtype=np.uint8), cv2.IMREAD_COLOR)
    
    movement = detect_camera_movement(im1, im2, 5)
    if movement:
        print("Camera movement detected")
        last_frame = frame
        results = model(unpack_frame_img(last_frame))
        sub_fixations = create_socket(ctx, ip, ["fixations"])
    
    fixation_data = msgpack.unpackb(sub_fixations.recv_multipart()[1], raw=False)
    
    pprint.pprint(fixation_data['duration'])
    gaze = fixation_data["norm_pos"]
    obj = get_object_in_gaze(gaze, results)
    
    if obj and "HueLamp" in obj:
        switch_lamp_on()
        # br = abs((abs(gaze[0] - 0.5) * 200) - 100)
        # set_lamp_brightness(br)
        time.sleep(2)

In [ ]:
import msgpack
import time


subscriber = create_socket(ctx, ip)
last_frame = None
while True:
    # Receive a message
    data = subscriber.recv_multipart()
    topic = data[0].decode("utf-8")
    if topic == "frame.world":
        last_frame = unpack_frame_img(data)

    if topic == "fixations":
        # Process fixation message
        fixation_data = msgpack.unpackb(data[1], raw=False)
        print("Received fixation")
        
        if last_frame is None:
            # Wait for the next frame message
            while True:
                data = subscriber.recv_multipart()
                topic = data[0].decode("utf-8")
                if topic == "frame.world":
                    last_frame = unpack_frame_img(data)
                    break
        
        # Get the gaze point
        gaze = fixation_data["norm_pos"]
        # Get the detected objects in the frame
        results = model(last_frame)
        # Get the object in gaze
        obj = get_object_in_gaze(gaze, results)
        
        # Switch the lamp on if the object is a lamp
        if obj and "HueLamp" in obj:
            switch_lamp_on()
            time.sleep(2)
            subscriber = create_socket(ctx, ip)

In [4]:
# Function to process the frames and detect movement
def detect_camera_movement(prev_frame, curr_frame):
    # Convert frames to grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Parameters for optical flow (Lucas-Kanade method)
    lk_params = dict(winSize=(21, 21), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

    # Detect good features to track in the first frame
    prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **dict(maxCorners=100, qualityLevel=0.3, minDistance=7))

    # Calculate optical flow (track points from prev_frame to curr_frame)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_points, None, **lk_params)

    # Select good points (where the flow has been successfully found)
    good_prev_points = prev_points[status == 1]
    good_next_points = next_points[status == 1]

    # Calculate the displacement of points
    displacements = good_next_points - good_prev_points
    avg_displacement = displacements.mean(axis=0)

    # Direction based on average displacement
    dx, dy = avg_displacement
    direction = ""
    tr = 10
    if abs(dx) > abs(dy):  # Horizontal movement
        if dx > tr:
            direction = f"Right: {dx}"
        elif dx < -1 * tr:
            direction = f"Left: {dx}"
    else:  # Vertical movement
        if dy > tr:
            direction = f"Down: {dy}"
        elif dy < -1 * tr:
            direction = f"Up: {dy}"

    return direction

In [ ]:
from PIL import Image
import numpy as np
import cv2
import zmq


subscriber = create_socket_fame(ctx, ip)
data = subscriber.recv_multipart()
last_frame = cv2.imdecode(np.frombuffer(data[2], dtype=np.uint8), cv2.IMREAD_COLOR)
while True:
    # Receive a message
    data = subscriber.recv_multipart()
    topic = data[0].decode("utf-8")
    frame = cv2.imdecode(np.frombuffer(data[2], dtype=np.uint8), cv2.IMREAD_COLOR)
    movement_direction = detect_camera_movement(last_frame, frame)
    if movement_direction != "":
        print(f"Camera Movement: {movement_direction}")
    # last_frame = frame
    


In [14]:
import msgpack
import pprint

subscriber = create_socket(ctx, ip, ["notify.pupil_size_change_event"])
data = subscriber.recv_multipart()
gaze = msgpack.unpackb(data[1], raw=False)
pprint.pprint(gaze)

KeyboardInterrupt: 

In [8]:
import zmq
import json

# Configuration for Pupil Core's IPC (defaults to localhost and port 50020)
PUPIL_IP = "127.0.0.1"
PUPIL_PORT = "50020"

def listen_to_events():
    """Connects to Pupil Core and listens for pupil_size_change_event."""
    
    # Setup ZeroMQ context and SUB socket
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connect(f"tcp://{PUPIL_IP}:{PUPIL_PORT}")
    
    # Subscribe to all topics to filter in the code
    socket.setsockopt_string(zmq.SUBSCRIBE, "")

    print("Listening for pupil_size_change_event...")

    try:
        while True:
            # Receive messages from Pupil Core
            topic, payload = socket.recv_multipart()
            
            # Decode the message
            topic = topic.decode("utf-8")
            payload = json.loads(payload.decode("utf-8"))
            print(topic)
            
            # Check if the message is the pupil size change event
            if "pupil_size" in topic:
                print("Pupil Size Change Event Detected!")
                print(f"Change Detected: {payload['change_detected']}")
                print(f"Size Change Amount: {payload['size_change']}")

    except KeyboardInterrupt:
        print("Stopped listening.")
    finally:
        # Clean up
        socket.close()
        context.term()

listen_to_events()

Listening for pupil_size_change_event...
Stopped listening.


In [ ]:
import msgpack
import pprint

subscriber = create_socket_gaze(ctx, ip)

curr_d = 0
while True:
    ld = []
    for i in range(40):
        data = subscriber.recv_multipart()
        gaze = msgpack.unpackb(data[1], raw=False)
        ld.append(gaze["base_data"][0]['diameter'])
    
    # if the mean of the last 20 diameters is greater than the current diameter
    diff = sum(ld) / len(ld) - curr_d
    if diff > 1:
        print("Diameter is increasing")
        curr_d = sum(ld) / len(ld)
    elif diff < -1:
        print("Diameter is decreasing")
        curr_d = sum(ld) / len(ld)
    
    subscriber = create_socket_gaze(ctx, ip)
    
    
    


In [ ]:
import requests

# Define the URL of the lamp
url = "http://10.2.2.33:1880/r402/theglobe"

# Define the payload to turn the lamp off
payload = {
    "on": True,  # Turn the lamp off
    "brightness": 50
}

# Send the PUT request to change the state of the lamp
try:
    response = requests.put(url, json=payload)
    if response.status_code == 200:
        print("The lamp has been turned off successfully.")
    else:
        print(f"Failed to turn off the lamp. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    
# Sent a GET request to retrieve the Thing Description
response = requests.get(url)
print(response.json())